In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [9]:
def canny(img):

    canny = cv.Canny(img, 50, 150)
    return canny

In [10]:
def region_of_interest(img):
    height = img.shape[0]
    poly = np.array([[(200,height), (1200, height), (790,450), (580, 450)]])
    mask = np.zeros_like(img)
    cv.fillPoly(mask, poly, 255)
    return mask

In [12]:
cap = cv.VideoCapture('project_video.mp4')
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    # Display the resulting frame
    gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    blur = cv.GaussianBlur(gray, (3,3), 0)
    interestt = blur & region_of_interest(blur)
    canny_img = canny(interestt)

    
    cv.imshow('Frame',canny_img)
    
    # Press Q on keyboard to  exit
    if cv.waitKey(25) & 0xFF == ord('q'):
      break

  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv.destroyAllWindows()